# 束搜索
:label:`sec_beam-search`

在 :numref:`sec_seq2seq` 中，我们介绍了编码器-解码器架构以及端到端训练它们的标准技术。然而，在测试时预测中，我们仅提到了*贪心*策略，即在每个时间步选择具有最高预测概率的下一个令牌，直到某个时间步我们预测到特殊的序列结束"&lt;eos&gt;"令牌。在本节中，我们将首先形式化这种*贪心搜索*策略，并识别实践中常遇到的一些问题。随后，我们将比较两种替代策略：*穷举搜索*（说明性的但不实用）和*束搜索*（实际中的标准方法）。

让我们从建立数学符号开始，借用 :numref:`sec_seq2seq` 中的惯例。在任何时间步 $t'$，解码器输出表示词汇表中每个令牌作为序列中下一个令牌的概率（$y_{t'+1}$ 的可能值），条件是之前的令牌 $y_1, \ldots, y_{t'}$ 和由编码器生成以表示输入序列的上下文变量 $\mathbf{c}$。为了量化计算成本，用 $\mathcal{Y}$ 表示输出词汇表（包括特殊的序列结束令牌 "&lt;eos&gt;"）。还指定输出序列的最大令牌数为 $T'$。我们的目标是从所有 $\mathcal{O}(\left|\mathcal{Y}\right|^{T'})$ 可能的输出序列中搜索理想的输出。请注意，这稍微高估了不同的输出数量，因为在出现 "&lt;eos&gt;" 令牌后没有后续令牌。然而，对于我们来说，这个数字大致捕捉到了搜索空间的大小。

## 贪心搜索

考虑 :numref:`sec_seq2seq` 中简单的*贪心搜索*策略。在这里，在任何时间步 $t'$，我们只需从 $\mathcal{Y}$ 中选择具有最高条件概率的令牌，即

$$y_{t'} = \operatorname*{argmax}_{y \in \mathcal{Y}} P(y \mid y_1, \ldots, y_{t'-1}, \mathbf{c}).$$

一旦我们的模型输出 "&lt;eos&gt;"（或达到最大长度 $T'$），输出序列就完成了。

这种策略看起来可能是合理的，事实上它也不算太差！考虑到它的计算需求非常低，你很难找到性价比更高的方法。然而，如果我们暂时抛开效率，可能会觉得寻找*最有可能的序列*更合理，而不是（贪心选择的）*最有可能的令牌*组成的序列。事实证明，这两个对象可以相当不同。最有可能的序列是使表达式 $\prod_{t'=1}^{T'} P(y_{t'} \mid y_1, \ldots, y_{t'-1}, \mathbf{c})$ 最大化的那个。在我们的机器翻译例子中，如果解码器真正恢复了潜在生成过程的概率，那么这将给我们提供最可能的翻译。不幸的是，不能保证贪心搜索会给出这个序列。

让我们用一个例子来说明。假设输出字典中有四个令牌 "A"、"B"、"C" 和 "&lt;eos&gt;"。在 :numref:`fig_s2s-prob1` 中，每个时间步下的四个数字分别代表在该时间步生成 "A"、"B"、"C" 和 "&lt;eos&gt;" 的条件概率。

![在每个时间步，贪心搜索选择具有最高条件概率的令牌。](../img/s2s-prob1.svg)
:label:`fig_s2s-prob1`

在每个时间步，贪心搜索选择具有最高条件概率的令牌。因此，输出序列 "A"、"B"、"C" 和 "&lt;eos&gt;" 将被预测（:numref:`fig_s2s-prob1`）。该输出序列的条件概率为 $0.5\times0.4\times0.4\times0.6 = 0.048$。

接下来，让我们看看 :numref:`fig_s2s-prob2` 中的另一个例子。与 :numref:`fig_s2s-prob1` 不同，在时间步 2 我们选择具有*第二*高条件概率的令牌 "C"。

![每个时间步下的四个数字代表在该时间步生成 "A"、"B"、"C" 和 "&lt;eos&gt;" 的条件概率。在时间步 2，选择具有第二高条件概率的令牌 "C"。](../img/s2s-prob2.svg)
:label:`fig_s2s-prob2`

由于时间步 3 所基于的时间步 1 和 2 的输出子序列从 :numref:`fig_s2s-prob1` 中的 "A" 和 "B" 变为 :numref:`fig_s2s-prob2` 中的 "A" 和 "C"，时间步 3 的每个令牌的条件概率也在 :numref:`fig_s2s-prob2` 中发生了变化。假设我们在时间步 3 选择令牌 "B"。现在时间步 4 依赖于前三个时间步的输出子序列 "A"、"C" 和 "B"，这已经从 :numref:`fig_s2s-prob1` 中的 "A"、"B" 和 "C" 改变。因此，:numref:`fig_s2s-prob2` 中时间步 4 生成每个令牌的条件概率也与 :numref:`fig_s2s-prob1` 中的不同。结果，:numref:`fig_s2s-prob2` 中输出序列 "A"、"C"、"B" 和 "&lt;eos&gt;" 的条件概率为 $0.5\times0.3 \times0.6\times0.6=0.054$，大于 :numref:`fig_s2s-prob1` 中贪心搜索的条件概率。在这个例子中，通过贪心搜索获得的输出序列 "A"、"B"、"C" 和 "&lt;eos&gt;" 并不是最优的。

## 穷举搜索

如果目标是获得最有可能的序列，我们可以考虑使用*穷举搜索*：枚举所有可能的输出序列及其条件概率，然后输出得分最高的预测概率的那个序列。

虽然这肯定会给我们想要的结果，但它将以 $\mathcal{O}(\left|\mathcal{Y}\right|^{T'})$ 的计算成本为代价，这个成本随着序列长度呈指数增长，并且基数是由词汇量大小决定的巨大数值。例如，当 $|\mathcal{Y}|=10000$ 和 $T'=10$ 时，这两个数字在实际应用中都相对较小，我们需要评估 $10000^{10} = 10^{40}$ 个序列，这已经超出了任何可预见计算机的能力。另一方面，贪心搜索的计算成本为 $\mathcal{O}(\left|\mathcal{Y}\right|T')$：奇迹般地便宜但远非最优。例如，当 $|\mathcal{Y}|=10000$ 和 $T'=10$ 时，我们只需要评估 $10000\times10=10^5$ 个序列。

## 束搜索

你可以将序列解码策略视为位于一个谱系上，其中*束搜索*在贪心搜索的效率和穷举搜索的最优性之间取得了折衷。最直接版本的束搜索由一个单一的超参数——*束大小* $k$ 来表征。让我们解释一下这个术语。在时间步 1，我们选择具有最高预测概率的 $k$ 个令牌。每个令牌将成为 $k$ 个候选输出序列的第一个令牌。在每个后续时间步，基于前一时间步的 $k$ 个候选输出序列，我们继续从 $k\left|\mathcal{Y}\right|$ 种可能的选择中选择具有最高预测概率的 $k$ 个候选输出序列。

![束搜索的过程（束大小 $=2$；输出序列的最大长度 $=3$）。候选输出序列为 $\mathit{A}$、$\mathit{C}$、$\mathit{AB}$、$\mathit{CE}$、$\mathit{ABD}$ 和 $\mathit{CED}$。](../img/beam-search.svg)
:label:`fig_beam-search`

:numref:`fig_beam-search` 演示了一个束搜索的例子。假设输出词汇表只包含五个元素：$\mathcal{Y} = \{A, B, C, D, E\}$，其中一个为“&lt;eos&gt;”。设束大小为 2，输出序列的最大长度为 3。在时间步 1，假设具有最高条件概率 $P(y_1 \mid \mathbf{c})$ 的令牌是 $A$ 和 $C$。在时间步 2，对于所有的 $y_2 \in \mathcal{Y}$，我们计算

$$\begin{aligned}P(A, y_2 \mid \mathbf{c}) = P(A \mid \mathbf{c})P(y_2 \mid A, \mathbf{c}),\\ P(C, y_2 \mid \mathbf{c}) = P(C \mid \mathbf{c})P(y_2 \mid C, \mathbf{c}),\end{aligned}$$

并从中挑选最大的两个值，比如 $P(A, B \mid \mathbf{c})$ 和 $P(C, E \mid \mathbf{c})$。然后在时间步 3，对于所有的 $y_3 \in \mathcal{Y}$，我们计算

$$\begin{aligned}P(A, B, y_3 \mid \mathbf{c}) = P(A, B \mid \mathbf{c})P(y_3 \mid A, B, \mathbf{c}),\\P(C, E, y_3 \mid \mathbf{c}) = P(C, E \mid \mathbf{c})P(y_3 \mid C, E, \mathbf{c}),\end{aligned}$$

并从中挑选最大的两个值，比如 $P(A, B, D \mid \mathbf{c})$ 和 $P(C, E, D \mid  \mathbf{c})$。最终，我们得到六个候选输出序列：(i) $A$；(ii) $C$；(iii) $A$，$B$；(iv) $C$，$E$；(v) $A$，$B$，$D$；和 (vi) $C$，$E$，$D$。

最后，我们根据这六个序列获得最终的候选输出序列集（例如，丢弃包含和之后的“&lt;eos&gt;”部分）。然后我们选择最大化以下分数的输出序列：

$$ \frac{1}{L^\alpha} \log P(y_1, \ldots, y_{L}\mid \mathbf{c}) = \frac{1}{L^\alpha} \sum_{t'=1}^L \log P(y_{t'} \mid y_1, \ldots, y_{t'-1}, \mathbf{c});$$
:eqlabel:`eq_beam-search-score`

这里 $L$ 是最终候选序列的长度，$\alpha$ 通常设置为 0.75。由于较长的序列在 :eqref:`eq_beam-search-score` 的求和中有更多的对数项，分母中的 $L^\alpha$ 项惩罚了长序列。

束搜索的计算成本为 $\mathcal{O}(k\left|\mathcal{Y}\right|T')$。这个结果介于贪心搜索和穷举搜索之间。贪心搜索可以被视为束大小设置为 1 时的束搜索特例。

## 总结

序列搜索策略包括贪心搜索、穷举搜索和束搜索。束搜索通过灵活选择束大小提供了准确性和计算成本之间的权衡。

## 练习

1. 我们能否将穷举搜索视为一种特殊类型的束搜索？为什么？
1. 在 :numref:`sec_seq2seq` 的机器翻译问题中应用束搜索。束大小如何影响翻译结果和预测速度？
1. 我们在 :numref:`sec_rnn-scratch` 中使用语言建模来生成跟随用户提供的前缀的文本。它使用哪种搜索策略？你能改进它吗？

[讨论](https://discuss.d2l.ai/t/338)